# Applied Data Science Capstone - Battle of the Cities

## Table of Contents

1. <a href="#item1">Introduction: Problem and Background</a>    
2. <a href="#item2">Data</a> 
3. <a href="#item3">Methodology</a>
4. <a href="#item4">Summary</a>


<a id="#item1"></a>
## Introduction

In the labs for Week 3, we examined how the neighborhoods of a large metropolitan city (New York or Toronto) could be segmented according to the types of business locations found in each neigborhood. In this project, I would like to examine types of business across multiple major metropolitan areas across the United States.

### Battle of the Cities

I call this project 'Battle of the Cities'. The primary question I would like to answer is

* Can we differentiate the major metropolitan areas of the United states by looking at the types of locations, and the proportions of each type, within those cities?
* Can we derive meaningful neighborhood profiles that will distinguish one major city from another? 

I will examine various neighborhoods within multiple cities, and obtain the locations of interest within each neighborhood using the FourSquare API. Each 'location of interest' is categorized by the FourSquare API into one of about 2,000 categories. So I can use the proportion of each category to make a statistical profile of each neighborhood.

### Background

Many people would like to understand what are the characteristics of the various big cities in the U.S. Interested parties would certainly include people who are relocating, and who would like to settle in a city which would be most agreeable to their particular life style and tastes. But the characterizations can also be of use to 

* Realtors, who have clients that are moving from one area of the country to another
* Marketing professionals, who would like to target advertising to each city. If the characteristics of each city are understood, marketing campaigns can be tailored to each city, and thereby made more effective
* City Planners, who would like to understand the unique characteristics of each city
* Small Business owners, who would like to establish a new business in a particular city

<hr>
<a id="#item2"></a>

### Data

There are two pieces of data which are required. The first will be a selection of cities, which will be chosen manually. I will choose about 10 cities from major metropolitan areas in the U.S.

The second piece of data will be a selection of the locations returned from the FourSquare API. We will collect 25 to 50 neighborhoods from each city, each within 5 miles of the city center. The centers of each neighborhood will be chosen randomly, and the neighborhood will be defined as locations within 500 meters of the randomly chosen point.

This will yield somewhere between 175 to 500 neighborhoods, scattered across the chosen major metropolitan areas. This should be enough to perform a clustering analysis (probably k-means clustering like in the labs).

#### Metropolitan Areas

The major cities for this experiment were chosen manually. When I first set out to do this, I wanted to specifically include some cities which might be unique

* <em>Honolulu HI</em> and <em>Anchorage AK</em>, which are not part of the continental US
* <em>Detroit MI</em> which has undergone significant demographic changes in the past few decades, as the 
* <em>Des Moines, IA</em> which is a smaller major metropolitan area in the middle part of the country
* <em>Los Angeles and San Francisco CA</em> two cities in the same state, both on the Pacific Ocean, but which should have their own unique characteristics.

A the time of this writing (before I begin collecting data), my list of cities is:
* New York, NY
* Detroit, MI
* New Orleans, LA
* Honolulu, HI
* San Francisco, CA
* Los Angeles, CA
* Des Moines, IA
* Anchorage, AK
* Chicago, IL
* Dallas, TX
* Philadelphia, PA

#### Geocoding

Once the cities are selected, the first step will be to complete geocoding. Fortunately, there is a Wikipedia page for that

https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population

This page lists all cities in the U.S. with an estimated 2018 population greater than 100,000 people. All of the cities I selected for this project are included in a table found on the page, which fortunately also includes the longitude and latitude. Longitude and latitude will have to be parsed, but that is easily accomplished with basic Python string functions.

The next geocoding task is to determine, for each city, the boundaries (latitude and longitude) for an approximately 5 mile square around the center of the city. We want to pick random neighborhoods within this 5 mile square for our sample data.

First, let us compute the minimum and maximum latitude, given the latitude of a city. This is the simplest problem, because latitude lines are always the same distance from one another. One degree of latitude is always about 69 miles, so we will use 
<code>
    maxLat = lat + 69 / 5
    minLat = lat - 69 / 5
</code>
Determining the minimum and maximum longitude is only a little harder. Longitude lines get closer together as you get closer to the poles, and the distance between two longitude lines can be computed as 69 miles * cos( latitude). So
<code>
    maxLong = long + 69 * cos( lat) / 5
    minLong = long - 69 * cos( lat) / 5
 </code>
 When we do this in Python, we will have to convert the geo-coordinates from degrees to radians, but the task is simple enough. 
 
 What we have calculated is an approximate square, where each side of the square is about 10 miles. Exactly 10 miles on the vertical edges, a little longer than 10 miles on the southern edge, and a little shorter than 10 miles on the northern edge. We will choose random points within this square for our profiled neighborhoods.


#### Neighborhood Profile

For each neighborhood, we will use the FourSquare API to build a neighborhood profile. A 'neighborhood' will be centered at a random location within about 5 miles of the city center. The raw data for the neigbhorhood profile will simply be a list of one to 500 business categories that are found within a 500 meter radius of the selected. (We can adjust the radius of the area to be considered a neighborhood, using the FourSquare API).

Our raw data is then consisting of

| <p align="left">Attribute      | <p align="left">Description      |
| :----------------------------- | :------------------ |
| <p align="left">City           | <p align="left">Name of city. This attribute is to be predicted |
| <p align="left">Neighborhood   | <p align="left">An index from 1 to 50, of the neighborhood within the city   |
| <p align="left">Location Category       | <p align="left">Category of business within the neighborhood |
    
Each element represents a location returned by the FourSquare API. We would like to collect 500 locations around each neighborhood, and we will choose 50 neighborhoods within each city. There are 11 chosen cities, so we might have as many as 275,000 data points to process. However, some of the random points which serve as neighborhood centers will not contain the full 50 locations.



<hr>
<a id="#item3"></a>

### Methodology

#### Problem with Categorical Data

The data collected from FourSquare that we will use is 'categorical' -- it expresses a category which is the type of location rather than a real number such as total number of visitors or annual sales. The categories come in as string data, which is the name of the type of business, such as 'Pizza Place' or 'Coffee Shop'.

Categorical data is perfectly fine for building a training set. Classification algorithms will want each attribute to be transformed into a number, but that is easily accomplished with Python library APIs such as <code>sklearn.preprocessing.LabelEncoder()</code> or <code>pandas.get_dummies()</code>. These library functions will transform our categorical data into a numerical value. The <code>LabelEncoder</code> call will assign each category a numerical index, so we might find that 'Pizza Place' becomes 1, 'Coffee Shop' becomes 2, 'Hiking Trail' becomes 3, etc.

But categorical data like this is still very bad for classification. Applying the normal metric in the above example, we see that 'Coffee Shop' is similar to 'Pizza Place', but is also similar to 'Hiking Trail', because the numerical index assigned to 'Coffee Shop' is close to the index assigned to the other two categories.

There is no good way to assign a number to location categories in such a way that similar categories get numbers which are close together. Fortunately, there is a better technique to handle categorical data than simply encoding with <code>LabelEncoder()</code>. It is "one-hot" encoding, which is accomplished with <code>pandas.get_dummies()</code>

In one-hot encoding, our dataset which has a single categoriecal column for the location category will be transformed into a dataset having about 2000 columns, one column for each category. (2000 is the number of possible location categories that can be returned by the FourSquare API). The column names will be the location category. The column will be populated with 1 when the sample category matches the column name. Otherwise the column will be populated with 0.

Now we can pre-process the data a little bit to roll-up and count the number of samples for each neighborhood. To normalize the sample, we should really use a percentage of the total number of locations within the neighborhood instead of the count. So our list of 275,000 neighborhood + location becomes a dataset of about 25,000 rows, one row for each neighborhood. Each row will contain 2,000 columns (one column for each possible location returned by FourSquare), and the value will be the percentage of locations within that neighborhood which match a particular category.

We have moved the categorical nature of the data from the Location value returned by FourSquare to the columns of the dataset. Each column is no longer categorical, but expresses the probability that a given location category is present in each neighborhood.

But now we suffer from another problem:

#### Curse of Dimensionality

There are too many dimensions. Most categorical algorithms do not work well with datasets that have high dimensionality. We have transformed our sample data from a dataset that contains a single categorical column into a dataset that no longer has categorical data in any column, but will have approximately 2,000 columns.

Datasets with such a high dimensionality are not usually very good candidates for clustering algorithms such as k-Means. The problem is that there is a lot of room in high-dimensional hyperspace. The sample points get further away from each other as the dimension of the sample space grows, until all points are nearly the same distance from each other.

Here is an interesting paper describing the phenomenon
Raul Rojas. *The Curse of Dimensionality*. PDF file. February 15, 2015
https://www.inf.fu-berlin.de/inst/ag-ki/rojas_home/documents/tutorials/dimensionality.pdf

#### Solving the Curse

In the class lab (Segmenting and Clustering Neighborhoods in New York City), the Curse of Dimensionality was not addressed. When I ran the lab, I found 337 different category locations for Manhatten, and the proportion of each category was fed in to K-Means as a 337 x 40 dataset. But when you run the lab, you might get slightly different results because the values returned from the Four Square API will vary over time.

In my project I suspect that I will find more categories of data, for two reasons. First, I will be collecting more points. If I collect 20 neighborhoods in 11 chosen cities, I will have 220 rows (not 40). And second, because my neighborhoods are distributed across the U.S., I suspect I will find some categories that appear in one city but not in the other.

One approach to be used in dealing with dimensionality would be to simply remove some of the features. You could remove the features that were least common, on the basis that least common features are less important. But I think that when you compare neighborhoods, some of the most interesting attributes may not be the most common. In other words, every city will have a lot of 'Pizza Place' and 'Coffee Shop' locations. But maybe some of the less common categories such as 'Beach', 'Hiking Trail', or 'Historical Site' might be useful to distinguish the character of a set of neighborhoods.

A more advanced technique which can deal with dimensionality would be to pre-process the data with a dimension reducing algorithm such as PCA (Principal Component Analysis). You can target the number of dimensions to come out of the PCA classifier, and the algorithm will fit the data by combining those features which are most nearly dependant on one another.

Once the clustering is complete, we can compare the relative probability of each cluster by city. My hope is that we will see some clusters that are more common in some cities than in others. For example, neighborhoods with common 'Pizza Place' are common in Chicago and New York, but not so much in Honolulu or DesMoines.

<hr>
<a id="#item4"></a>

### Summary

I plan to do the following steps

1. Extract the latitude and longitude of about 12 major U.S. Cities from Wikipedia. Compute the minimum and maximum latitude and longitude for each city, to enclose an area of points within 5 miles of the center of the city.
2. Define about 50 random points within each of these 10 mile squares, which will be the center of a neighborhood and will be used as a parameter to invoke the FourSquare API
3. Invoke the FourSquare API to return up to 500 venues for each neighborhood. We will collect the Category of each returned Venue.
4. For each neigbborhood, summarize the probability of each category. There could be up to 2,000 categories, so each neighborhood profile would consist of a vector of up to 2,000 probabilities -- one probability for each possible venue category
5. Use a dimension reducing algorithm such as PCA, to reduce the number of features.
6. Apply k-Means clustering to segregate the data into some number of categories. This will result in a small number of neighborhood profile classes. 
7. Plot the results in (perhaps) a bar-chart, which will show each city and the relative proportion of each class of neighborhood profile. We might find that all the bars are nearly equal in height (which means that each city has a similar distribution of neighborhood profiles). Or we might find that some cities have a particular profile which is more common than in other cities.

Whether or not there is a difference from one city to another, we can also look at what lands in each neighborhood profile. 